In [ ]:
pwd

In [ ]:
import re
import pandas as pd

In [ ]:
f= open('Sample-Chats\WhatsApp Chat with BCA, 4 Sem Haldwani.txt', 'r', encoding= 'utf-8')

In [ ]:
data= f.read()

In [ ]:
print(type(data))

In [ ]:
print(data)

In [ ]:
pattern= '\d{1,2}/\d{1,2}/\d{1,2}, \d{1,2}:\d{1,2} \s?(?:a|p)\.?m\.? - +'

In [ ]:
messages= re.split(pattern
                   , data)[1:]

In [ ]:
messages

In [ ]:
dates= re.findall(pattern, data)
dates

In [ ]:
# for i in range(len(dates)):
#     dates[i]= dates[i].replace('a', 'A')
#     dates[i]= dates[i].replace('p', 'P')
#     dates[i]= dates[i].replace('m', 'M')
#     print(dates[i])

In [ ]:
# dates

In [ ]:
df= pd.DataFrame({'user_messsage': messages, 'message_date': dates})

In [ ]:
df['message_date']= pd.to_datetime(df['message_date'], format= "%d/%m/%y, %I:%M %p - ")

In [ ]:
df.head(10)

In [ ]:
df.shape

In [ ]:
df.rename(columns= {'message_date': 'date'}, inplace= True)
df.head(10)

In [ ]:
lstU=[] # list for users
lstM=[] # list for messages
msg= "Aanchal BCA changed this group's icon"
for msg in df['user_messsage']:
    entry= re.split('([\w\W]+?):\s', msg)
    print(entry)

    if entry[1:]:
        lstU.append(entry[1])
        lstM.append(entry[2])
    else:
        lstU.append('group_notification')
        lstM.append(entry[0])

# print(lstU)
# print(lstM)

In [ ]:
# users= []
# messages= []
# for message in df['user_messsage']:
#     entry= re.split('/([\w\W]+?):\s', message)
#     print(entry)
#     if entry[1:]:
#         users.append(entry[1])
#         messages.append(entry[2])
#     else:
#         users.append('group_notification')
#         messages.append(entry[0])



In [ ]:
lstU

In [ ]:
df['users']= lstU
df['message']= lstM

In [ ]:
df.drop(columns= ['user_messsage'], inplace= True)
df.head()

In [ ]:
# df['date'] # all dates 

In [ ]:
df['year']= df['date'].dt.year # get years from date
df.head()

In [ ]:
df['month']= df['date'].dt.month_name() # get the name of months
df.head()

In [ ]:
df['day']= df['date'].dt.day # get the day
df['hour']= df['date'].dt.hour # get the hour
df['minute']= df['date'].dt.minute # get the minute

df.head(20)


In [ ]:
words= []
for message in df['message']:
    # print(message)
    words.extend(message.split())
len(words)

In [ ]:
from urlextract import URLExtract
extractor= URLExtract()
urls= extractor.find_urls("I have google.com, bing.com and sometimes I work with drive.com")
urls

In [ ]:
links= []
for message in df['message']:
    # print(extractor.find_urls(message))
    links.extend((extractor.find_urls(message)))
links

In [ ]:
len(links)

In [ ]:
x= df['users'].value_counts().head()
x

In [ ]:
import matplotlib.pyplot as plt

name= x.index
count= x.values

In [ ]:
plt.bar(name, count)
plt.xticks(rotation= 'vertical')
plt.show()

In [ ]:
# percentage how much every user participated 
x = round((df['users'].value_counts()/df.shape[0])*100, 2) 
x

In [ ]:
x=x.reset_index() # dataframe
x

In [ ]:
x= x.rename(columns={'index': 'names', 'users': 'percentage'})
x

In [ ]:
round((df['users'].value_counts()/df.shape[0])*100,2).reset_index().rename(columns={'index': 'names', 'users': 'percentage'})


In [ ]:
df[df['users']=='group_notification']

In [ ]:
# Removing Group Notifications and Media Messages !!!

temp= df[df['users']!='group_notification']
temp = temp[temp['message'] != '<Media omitted>\n']
temp

In [ ]:
# removing stop words

f= open('hinglish.txt', 'r')
stop_words= f.read()
print(stop_words)

In [ ]:
words = []
for message in temp['message']:
    for word in message.lower().split():
        if word not in stop_words:
            words.append(word)


In [ ]:
from collections import Counter
pd.DataFrame(Counter(words).most_common(20))


In [ ]:
words= []
for message in df['message']:
    words.extend(message.split())

In [ ]:
from collections import Counter
pd.DataFrame(Counter(words).most_common(20))

In [ ]:
def most_common_words(selected_user, df):
    if selected_user != 'Overall':
        df = df[df['users'] == selected_user]

    # Removing Group Notifications and Media Messages !!!
    temp = df[df['users'] != 'group_notification']
    temp = temp[temp['message'] != '<Media omitted>\n']
    temp = temp[temp['message'] != '<media omitted>']

    # removing stop words
    f = open('hinglish.txt', 'r')
    stop_words = f.read()
    words = []
    for message in temp['message']:
        print(message)
        for word in message.lower().split():
            if word not in stop_words:
                words.append(word)
    most_common_df = pd.DataFrame(Counter(words).most_common(20))
    return most_common_df


In [ ]:
most_common_words('Overall', df)


In [ ]:
df.head()

In [ ]:
import emoji

In [ ]:
emojis= []
for message in df['message']:
    emojis.extend([c for c in message if c in emoji.UNICODE_EMOJI['en']])

In [ ]:
emojis

In [ ]:
df['month_num']= df['date'].dt.month
df.head()

In [ ]:
timeline= df.groupby(['year', 'month_num', 'month']).count()['message']
timeline

In [ ]:
timeline= timeline.reset_index()
timeline

In [ ]:
for i in range(timeline.shape[0]):
    print(timeline['month'][i]+ '-'+ str(timeline['year'][i]))